In [1]:
from syft.core.hooks import TorchHook
from syft.core.hooks import torch

hook = TorchHook()
x = torch.FloatTensor([1,2,3,4,5])
x

Hooking into Torch...
Overloading complete.


AttributeError: 'TorchHook' object has no attribute 'register_object_'

In [2]:
hook.local_worker

In [3]:
x.owners

[0]

In [1]:
from syft.core.worker import LocalWorker, BaseWorker

import torch.nn as nn
from torch.autograd import Variable as Var

hook = syft.core.torch_.hook.TorchHook(is_client=True)
local = hook.worker
remote = LocalWorker(id=1, hook=hook)

hook.add_worker(remote.id,remote)

AttributeError: 'TorchHook' object has no attribute 'client_worker'

In [2]:
x = Var(torch.ones(2,2),requires_grad=True).send_(remote)
x2 = Var(torch.ones(2,2),requires_grad=True).send_(remote)

In [3]:
x.backward(x2)

In [4]:
x.get_()

Variable containing:
 1  1
 1  1
[torch.FloatTensor of size 2x2]

In [5]:
x.grad

In [11]:
remote._objects

{1696728241: Variable containing:[torch.FloatTensor - Locations:[<syft.core.worker.LocalWorker object at 0x103ceafd0>]],
 4378825613: Variable containing:[torch.FloatTensor - Locations:[<syft.core.worker.LocalWorker object at 0x103ceafd0>]]}

In [13]:
remote._objects[4378825613].grad

Variable containing:
 1  1
 1  1
[torch.FloatTensor of size 2x2]